<a href="https://colab.research.google.com/github/Arnab1181412/Health-Insurance-Cross-Sell-Prediction/blob/main/Feature_Selection_and_Engineering_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

**Importing the Libraries**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import featuretools as ft

In [4]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [5]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv',index_col=0)
dataset.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [6]:
dataset = dataset.sample(frac=0.1,random_state=42)
dataset.shape

(38111, 11)

In [7]:
X = dataset.drop('Response',axis=1)
y = dataset.Response

**This is an Imbalanced dataset**

In [8]:
y.value_counts()/38111*100

0    87.449818
1    12.550182
Name: Response, dtype: float64

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38111 entries, 200223 to 101469
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Gender                38111 non-null  object 
 1   Age                   38111 non-null  int64  
 2   Driving_License       38111 non-null  int64  
 3   Region_Code           38111 non-null  float64
 4   Previously_Insured    38111 non-null  int64  
 5   Vehicle_Age           38111 non-null  object 
 6   Vehicle_Damage        38111 non-null  object 
 7   Annual_Premium        38111 non-null  float64
 8   Policy_Sales_Channel  38111 non-null  float64
 9   Vintage               38111 non-null  int64  
 10  Response              38111 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 3.5+ MB


Types of Variables

**0 Null values**

**Seperating different type of variables**

In [10]:
numerical_cols = [col for col in X.columns if X[col].dtypes != 'O']
numerical_cols

['Age',
 'Driving_License',
 'Region_Code',
 'Previously_Insured',
 'Annual_Premium',
 'Policy_Sales_Channel',
 'Vintage']

In [11]:
categorical_cols = [col for col in X.columns if X[col].dtypes == 'O']
categorical_cols

['Gender', 'Vehicle_Age', 'Vehicle_Damage']

In [12]:
continuous_numerical_cols = [col for col in numerical_cols if len(X[col].unique())>25]
continuous_numerical_cols.remove('Region_Code')
continuous_numerical_cols

['Age', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

In [13]:
discrete_numerical_cols = [col for col in numerical_cols if col not in continuous_numerical_cols]
discrete_numerical_cols

['Driving_License', 'Region_Code', 'Previously_Insured']

**OHE for Vehicle_Damage**

In [14]:
gender_ohe = pd.get_dummies(X['Gender'],drop_first=True)
vehicle_damage_ohe = pd.get_dummies(X['Vehicle_Damage'],drop_first=True,prefix='Vehicle_Damage')
X.drop(['Gender','Vehicle_Damage'],axis=1,inplace=True)


In [15]:
X = pd.concat([X,gender_ohe,vehicle_damage_ohe],axis=1)
X.head()

,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Annual_Premium,Policy_Sales_Channel,Vintage,Male,Vehicle_Damage_Yes
id,,,,,,,,,,
200223,21,1,3.0,1,< 1 Year,20408.0,160.0,72,0,0
49767,55,1,15.0,0,1-2 Year,37498.0,26.0,102,1,1
172202,41,1,3.0,0,1-2 Year,2630.0,26.0,90,0,1
160714,26,1,11.0,0,< 1 Year,2630.0,151.0,268,0,0
53273,51,1,40.0,0,1-2 Year,34006.0,124.0,265,1,1


**Manual Label Encoding for Vehicle_Age column (For ref :- See the EDA notebook)**

In [16]:
for i,j in enumerate(X['Vehicle_Age'].unique().tolist()[::-1]):
    X['Vehicle_Age'] = np.where(X['Vehicle_Age']==j,i+1,X['Vehicle_Age'])

In [17]:
X.head()

,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Annual_Premium,Policy_Sales_Channel,Vintage,Male,Vehicle_Damage_Yes
id,,,,,,,,,,
200223,21,1,3.0,1,3,20408.0,160.0,72,0,0
49767,55,1,15.0,0,2,37498.0,26.0,102,1,1
172202,41,1,3.0,0,2,2630.0,26.0,90,0,1
160714,26,1,11.0,0,3,2630.0,151.0,268,0,0
53273,51,1,40.0,0,2,34006.0,124.0,265,1,1


**Handling the Imbalanced dataset**

In [18]:
from  imblearn.combine import SMOTETomek
os = SMOTETomek(random_state=0)
X_os,y_os = os.fit_resample(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [19]:
X_os = pd.DataFrame(X_os,columns = X.columns)
y_os = pd.DataFrame(y_os)

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X_os,y_os,random_state=42,test_size=0.25)


In [21]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train = pd.DataFrame(scalar.fit_transform(X_train),columns = X_os.columns)
X_valid = pd.DataFrame(scalar.transform(X_valid),columns = X_os.columns)

In [22]:
X_train.head()

,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Annual_Premium,Policy_Sales_Channel,Vintage,Male,Vehicle_Damage_Yes
0,0.072242,0.033087,-0.709230,-0.599637,-0.465709,0.802011,-1.412777,-0.198075,-0.185449,0.648585
1,-0.548088,0.033087,0.447220,-0.599637,0.405275,-0.038342,0.909625,1.205091,-0.227414,0.648585
2,2.244438,0.033087,0.139391,-0.599637,-0.465709,1.046080,-1.412777,0.121647,0.932943,0.648585
3,-1.348478,0.033087,1.541922,-0.599637,1.401763,-0.648168,0.880489,-0.917956,0.932943,0.648585
4,-1.420336,0.033087,-0.933132,-0.599637,1.401763,-0.124437,0.880489,0.659794,0.932943,0.648585


In [23]:
from sklearn.feature_selection import VarianceThreshold
var_thres = VarianceThreshold(0)
var_thres.fit(X_train)
var_thres.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

**Creating New feautures using Deep Feature Synthesis (DFS)**

In [24]:
es = ft.EntitySet(id = 'health')
es.entity_from_dataframe(entity_id = 'hr', dataframe = X_train, index = 'index')

# Run deep feature synthesis with transformation primitives
feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'hr',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'], 
                                      verbose=True)

/content/drive/MyDrive/Colab Notebooks/featuretools/entityset/entity.py:463: UserWarning: index index not found in dataframe, creating new integer column
  "integer column".format(index))


Built 100 features
Elapsed: 00:00 | Progress:   0%|          

/content/drive/MyDrive/Colab Notebooks/featuretools/synthesis/deep_feature_synthesis.py:152: UserWarning: Only one entity in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn("Only one entity in entityset, changing max_depth to "


Elapsed: 00:00 | Progress: 100%|██████████


**Getting the columns whose corelation is less**

In [25]:
def correlation(df,threshold):
    data = df.copy()
    corr_columns = set()
    corr = data.corr()
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i,j])>threshold:
                corr_columns.add(corr.columns[i])
            
    return corr_columns

In [27]:
imp_columns = correlation(feature_matrix.copy(),0.85)
imp_columns

{'Age * Driving_License',
 'Male * Vehicle_Damage_Yes',
 'Policy_Sales_Channel + Vehicle_Age',
 'Previously_Insured * Vehicle_Damage_Yes',
 'Previously_Insured + Vehicle_Age',
 'Vehicle_Age * Vehicle_Damage_Yes',
 'Vehicle_Damage_Yes',
 'Vehicle_Damage_Yes * Vintage'}

In [28]:
feature_matrix  = feature_matrix[imp_columns]
feature_matrix.head()

,Previously_Insured * Vehicle_Damage_Yes,Previously_Insured + Vehicle_Age,Age * Driving_License,Policy_Sales_Channel + Vehicle_Age,Vehicle_Age * Vehicle_Damage_Yes,Vehicle_Damage_Yes * Vintage,Male * Vehicle_Damage_Yes,Vehicle_Damage_Yes
index,,,,,,,,
0,-0.388915,-1.065346,0.002390,-1.878486,-0.302052,-0.128468,-0.120279,0.648585
1,-0.388915,-0.194362,-0.018135,1.314900,0.262855,0.781604,-0.147497,0.648585
2,-0.388915,-1.065346,0.074262,-1.878486,-0.302052,0.078898,0.605093,0.648585
3,-0.388915,0.802126,-0.044618,2.282252,0.909162,-0.595372,0.605093,0.648585
4,-0.388915,0.802126,-0.046995,2.282252,0.909162,0.427932,0.605093,0.648585


In [29]:
X_train_red = feature_matrix
X_train_red.head()

,Previously_Insured * Vehicle_Damage_Yes,Previously_Insured + Vehicle_Age,Age * Driving_License,Policy_Sales_Channel + Vehicle_Age,Vehicle_Age * Vehicle_Damage_Yes,Vehicle_Damage_Yes * Vintage,Male * Vehicle_Damage_Yes,Vehicle_Damage_Yes
index,,,,,,,,
0,-0.388915,-1.065346,0.002390,-1.878486,-0.302052,-0.128468,-0.120279,0.648585
1,-0.388915,-0.194362,-0.018135,1.314900,0.262855,0.781604,-0.147497,0.648585
2,-0.388915,-1.065346,0.074262,-1.878486,-0.302052,0.078898,0.605093,0.648585
3,-0.388915,0.802126,-0.044618,2.282252,0.909162,-0.595372,0.605093,0.648585
4,-0.388915,0.802126,-0.046995,2.282252,0.909162,0.427932,0.605093,0.648585


**For X_valid**

In [30]:
es_valid = ft.EntitySet(id = 'health_test')
es_valid.entity_from_dataframe(entity_id = 'hr', dataframe = X_valid, index = 'index')

# Run deep feature synthesis with transformation primitives
feature_matrix_test, feature_defs_test = ft.dfs(entityset = es_valid, target_entity = 'hr',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'], 
                                      verbose=True)

Built 100 features
Elapsed: 00:00 | Progress: 100%|██████████


/content/drive/MyDrive/Colab Notebooks/featuretools/entityset/entity.py:463: UserWarning: index index not found in dataframe, creating new integer column
  "integer column".format(index))
/content/drive/MyDrive/Colab Notebooks/featuretools/synthesis/deep_feature_synthesis.py:152: UserWarning: Only one entity in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn("Only one entity in entityset, changing max_depth to "


In [ ]:
X_valid_red = feature_matrix_test[feature_matrix.columns]

**Using cross validation to see which model will perform best**

In [31]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from xgboost import  XGBClassifier
from sklearn.svm import SVC
from sklearn.tree import  DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def modelpicker(models):
  summary = {}
  for i in models:
    accuracy = cross_val_score(i,feature_matrix,y_train,scoring='accuracy',cv=5,n_jobs=-1).mean()
    summary[i] = accuracy
    print(summary)
  return summary


In [ ]:
models = [RandomForestClassifier(),AdaBoostClassifier(),SVC(),XGBClassifier(),DecisionTreeClassifier()]
modelpicker(models)

{RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False): 0.8882804397427918}
{RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
       

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



{RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False): 0.8882804397427918, AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None): 0.8589296826384567, SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False): 0.8233146650072598, XGBC

{AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                    n_estimators=50, random_state=None): 0.8589296826384567,
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best'): 0.8795892968263846,
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_wei

**XGboost has highest training accuracy ie; 0.9159510475005185**

**HyperParameter Tuning**

In [ ]:
import optuna
from xgboost import XGBClassifier
def objective(trial):
  learning_rate = trial.suggest_float('learning_rate',0.05,0.3,step = 0.05)
  max_depth = trial.suggest_int('max_depth',3,20,1)
  min_child_weight = trial.suggest_int('min_child_weight',1,8,2) 
  gamma = trial.suggest_float('gamma',0.0,0.5,step = 0.1)
  colsample_bytree = trial.suggest_float('colsample_bytree',0.3,0.9,step = 0.1)
  n_estimators = trial.suggest_int('n_estimators',200,1000,50)
  clf = XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                      max_depth = max_depth,min_child_weight = min_child_weight,
                      gamma = gamma,colsample_bytree = colsample_bytree)
  return cross_val_score(clf,feature_matrix,y_train,n_jobs=-1,cv = 3).mean()
  

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials = 100)


[I 2021-07-03 08:29:22,098] A new study created in memory with name: no-name-bcb73b87-a06e-4f39-9ff5-49d38de7d885
/content/drive/MyDrive/Colab Notebooks/optuna/distributions.py:563: UserWarning:

The distribution is specified by [1, 8] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 7].

[I 2021-07-03 08:29:38,010] Trial 0 finished with value: 0.9112217382285833 and parameters: {'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 7, 'gamma': 0.4, 'colsample_bytree': 0.5, 'n_estimators': 300}. Best is trial 0 with value: 0.9112217382285833.
/content/drive/MyDrive/Colab Notebooks/optuna/distributions.py:563: UserWarning:

The distribution is specified by [1, 8] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 7].

[I 2021-07-03 08:30:51,039] Trial 1 finished with value: 0.9082970338104128 and parameters: {'learning_rate': 0.2, 'max_depth': 11, 'min_child_weight': 3, 'gamma': 0.2, 'colsample_bytree': 0.8, 'n_estimator

AttributeError: ignored

In [ ]:
trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.9207425845260321
Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'gamma': 0.5, 'colsample_bytree': 0.3, 'n_estimators': 900}


**Now we will fit the whole dataset**

In [32]:
full_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv',index_col=0)

In [33]:
X_f = full_dataset.drop('Response',axis=1)
y_f = full_dataset.Response

In [34]:
numerical_cols = [col for col in X_f.columns if X_f[col].dtypes != 'O']
numerical_cols

['Age',
 'Driving_License',
 'Region_Code',
 'Previously_Insured',
 'Annual_Premium',
 'Policy_Sales_Channel',
 'Vintage']

In [35]:
categorical_cols = [col for col in X_f.columns if X_f[col].dtypes == 'O']
categorical_cols

['Gender', 'Vehicle_Age', 'Vehicle_Damage']

In [36]:
continuous_numerical_cols = [col for col in numerical_cols if len(X_f[col].unique())>25]
continuous_numerical_cols.remove('Region_Code')
continuous_numerical_cols

['Age', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

In [37]:
discrete_numerical_cols = [col for col in numerical_cols if col not in continuous_numerical_cols]
discrete_numerical_cols

['Driving_License', 'Region_Code', 'Previously_Insured']

In [38]:
gender_ohe = pd.get_dummies(X_f['Gender'],drop_first=True)
vehicle_damage_ohe = pd.get_dummies(X_f['Vehicle_Damage'],drop_first=True,prefix='Vehicle_Damage')
X_f.drop(['Gender','Vehicle_Damage'],axis=1,inplace=True)


In [39]:
X_f = pd.concat([X_f,gender_ohe,vehicle_damage_ohe],axis=1)
X_f.head()

,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Annual_Premium,Policy_Sales_Channel,Vintage,Male,Vehicle_Damage_Yes
id,,,,,,,,,,
1,44,1,28.0,0,> 2 Years,40454.0,26.0,217,1,1
2,76,1,3.0,0,1-2 Year,33536.0,26.0,183,1,0
3,47,1,28.0,0,> 2 Years,38294.0,26.0,27,1,1
4,21,1,11.0,1,< 1 Year,28619.0,152.0,203,1,0
5,29,1,41.0,1,< 1 Year,27496.0,152.0,39,0,0


In [40]:
for i,j in enumerate(X_f['Vehicle_Age'].unique().tolist()[::-1]):
    X_f['Vehicle_Age'] = np.where(X_f['Vehicle_Age']==j,i+1,X_f['Vehicle_Age'])

In [41]:
from  imblearn.combine import SMOTETomek
os = SMOTETomek(random_state=33)
X_os_f,y_os_f = os.fit_resample(X_f,y_f)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [42]:
X_os_f = pd.DataFrame(X_os_f,columns = X_f.columns)
y_os_f = pd.DataFrame(y_os_f)

In [43]:
from sklearn.model_selection import train_test_split
X_train_full,X_valid_full,y_train_full,y_valid_full = train_test_split(X_os_f,y_os_f,random_state=42,test_size=0.25)


In [44]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train_full = pd.DataFrame(scalar.fit_transform(X_train_full),columns = X_os_f.columns)
X_valid_full = pd.DataFrame(scalar.transform(X_valid_full),columns = X_os_f.columns)

In [45]:
X_train_full.head()

,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Annual_Premium,Policy_Sales_Channel,Vintage,Male,Vehicle_Damage_Yes
0,-1.278778,0.041515,-0.792993,1.669788,-1.396623,0.057681,0.874099,1.689196,-1.238686,-1.562978
1,-1.350975,0.041515,-1.949684,1.669788,-1.396623,-0.504496,0.874099,0.138847,0.933964,-1.562978
2,-0.284657,0.041515,-1.002772,-0.600287,0.452360,-0.628865,1.044705,-0.515408,0.933964,0.648316
3,0.165165,0.041515,1.933492,-0.600287,0.452360,-0.488484,-1.392671,0.835903,0.933964,0.648316
4,-0.123623,0.041515,1.933492,1.669788,0.452360,1.154821,-1.716495,-1.471594,0.933964,-1.562978


In [46]:
es = ft.EntitySet(id = 'health_full')
es.entity_from_dataframe(entity_id = 'hr_full', dataframe = X_train_full, index = 'index')

# Run deep feature synthesis with transformation primitives
feature_matrix_full, feature_defs_full = ft.dfs(entityset = es, target_entity = 'hr_full',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'], 
                                      verbose=True)

Built 100 features
Elapsed: 00:00 | Progress:   0%|          

/content/drive/MyDrive/Colab Notebooks/featuretools/entityset/entity.py:463: UserWarning: index index not found in dataframe, creating new integer column
  "integer column".format(index))
/content/drive/MyDrive/Colab Notebooks/featuretools/synthesis/deep_feature_synthesis.py:152: UserWarning: Only one entity in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn("Only one entity in entityset, changing max_depth to "


Elapsed: 00:03 | Progress: 100%|██████████


In [47]:
feature_matrix_full = feature_matrix_full[feature_matrix.columns]
feature_matrix_full.head()

,Previously_Insured * Vehicle_Damage_Yes,Previously_Insured + Vehicle_Age,Age * Driving_License,Policy_Sales_Channel + Vehicle_Age,Vehicle_Age * Vehicle_Damage_Yes,Vehicle_Damage_Yes * Vintage,Male * Vehicle_Damage_Yes,Vehicle_Damage_Yes
index,,,,,,,,
0,-2.609842,0.273165,-0.053088,-0.522523,2.182891,-2.640177,1.936039,-1.562978
1,-2.609842,0.273165,-0.056085,-0.522523,2.182891,-0.217014,-1.459765,-1.562978
2,-0.389176,-0.147927,-0.011817,1.497066,0.293272,-0.334147,0.605503,0.648316
3,-0.389176,-0.147927,0.006857,-0.940311,0.293272,0.541929,0.605503,0.648316
4,-2.609842,2.122148,-0.005132,-1.264135,-0.707029,2.300069,-1.459765,-1.562978


In [48]:
es_valid = ft.EntitySet(id = 'health_test_full')
es_valid.entity_from_dataframe(entity_id = 'hr_test_full', dataframe = X_valid_full, index = 'index')

# Run deep feature synthesis with transformation primitives
feature_matrix_test_full, feature_defs_test_full = ft.dfs(entityset = es_valid, target_entity = 'hr_test_full',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'], 
                                      verbose=True)

Built 100 features
Elapsed: 00:00 | Progress:   0%|          

/content/drive/MyDrive/Colab Notebooks/featuretools/entityset/entity.py:463: UserWarning: index index not found in dataframe, creating new integer column
  "integer column".format(index))
/content/drive/MyDrive/Colab Notebooks/featuretools/synthesis/deep_feature_synthesis.py:152: UserWarning: Only one entity in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn("Only one entity in entityset, changing max_depth to "


Elapsed: 00:01 | Progress: 100%|██████████


In [49]:
feature_matrix_test_full = feature_matrix_test_full[feature_matrix_full.columns]

In [50]:
clf = XGBClassifier(learning_rate= 0.1, max_depth= 7, min_child_weight= 1, gamma= 0.5, colsample_bytree= 0.3, n_estimators= 900)
clf.fit(feature_matrix_full,y_train_full)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0.5,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=900, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [51]:
y_pred = clf.predict(feature_matrix_test_full)

In [52]:
from sklearn.metrics import classification_report,accuracy_score
score = accuracy_score(y_valid_full,y_pred)
score

0.9257470348463024

In [59]:
from sklearn.metrics import classification_report
report  = classification_report(y_valid_full,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.87      1.00      0.93     81451
           1       1.00      0.86      0.92     81694

    accuracy                           0.93    163145
   macro avg       0.93      0.93      0.93    163145
weighted avg       0.93      0.93      0.93    163145



In [54]:
import pickle
pickle.dump(clf, open("model.pkl", "wb"))

In [55]:
loaded_model = pickle.load(open("/content/model.pkl", "rb"))

In [56]:
y_pred = loaded_model.predict(feature_matrix_test_full)

In [57]:
from sklearn.metrics import classification_report,accuracy_score
score = accuracy_score(y_valid_full,y_pred)
score

0.9257470348463024